In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression # swap to Ridge(alpha=1.0) if needed
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

In [102]:
df = pd.read_csv("./data/PRSA_Data_Aotizhongxin_20130301-20170228.csv")
print(df.shape)
df.head()

(35064, 18)


,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin


In [103]:
df["dt"] = pd.to_datetime(df[["year", "month", "day", "hour"]])
df = df.sort_values("dt").set_index("dt")
df = df.drop(columns=["No","station"])  # single-station data
df.head()

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM
dt,,,,,,,,,,,,,,,,
2013-03-01 00:00:00,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4
2013-03-01 01:00:00,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7
2013-03-01 02:00:00,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6
2013-03-01 03:00:00,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1
2013-03-01 04:00:00,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0


In [ ]:
# Define pollutants and meteorological variables
pollutants = ["PM2.5", "PM10", "SO2", "NO2", "CO", "O3"]
meteo = ["TEMP", "PRES", "DEWP", "RAIN", "wd", "WSPM"]

# Calendar features
df["hour_sin"] = np.sin(2*np.pi*df.index.hour/24)
df["hour_cos"] = np.cos(2*np.pi*df.index.hour/24)
df["dow"] = df.index.dayofweek
df["month"] = df.index.month

# Helper to create lag/rolling features for a specific target
def add_lag_feats(frame, target, lag_hours=(1, 3, 6, 12, 24), roll_hours=(6, 24)):
    out = frame.copy()
    for h in lag_hours:
        out[f"{target}_lag{h}"] = out[target].shift(h)
    for w in roll_hours:
        out[f"{target}_roll{w}"] = out[target].rolling(w, min_periods=w).mean()
    return out

# Train/eval one pollutant
def fit_one_pollutant(target, df):
    # Build lagged dataset
    data = add_lag_feats(df, target)

    # Feature set:
    # - meteorology (with 'wd' categorical)
    # - calendar (hour_sin, hour_cos, dow, month)
    # - (optional) other pollutants as exogenous predictors — useful but watch leakage only if contemporaneous:
    #   You can include other pollutants *lagged* to avoid same-time leakage. Below we keep them contemporaneous off for strictness.
    feature_cols = meteo + ["hour_sin", "hour_cos", "dow", "month"] \
                   + [f"{target}_lag{h}" for h in (1,3,6,12,24)] \
                   + [f"{target}_roll{w}" for w in (6,24)]
    
    # Drop early rows with NaNs from lags/rolls
    model_df = data[feature_cols + [target]].dropna().copy()

    # Train/test split by time (last 20% as test)
    n = len(model_df)
    split = int(n * 0.8)
    train, test = model_df.iloc[:split], model_df.iloc[split:]

    X_train, y_train = train[feature_cols], train[target]
    X_test,  y_test  = test[feature_cols],  test[target]

    # Preprocess: one-hot for 'wd', scale numeric (especially helpful if you later use Ridge/Lasso)
    cat_cols = ["wd"]
    num_cols = [c for c in feature_cols if c not in cat_cols]

    pre = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
            ("num", StandardScaler(with_mean=False), num_cols)  # with_mean=False works with sparse mix
        ],
        remainder="drop"
    )

    pipe = Pipeline(steps=[
        ("prep", pre),
        ("ols", LinearRegression())
    ])

    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_test)

    mae = mean_absolute_error(y_test, pred)
    rmse = np.sqrt(mean_squared_error(y_test, pred))

    return {"target": target, "model": pipe, "MAE": mae, "RMSE": rmse,
            "y_true": y_test, "y_pred": pd.Series(pred, index=y_test.index)}

# Run for all pollutants
results = []
for p in pollutants:
    if p in df.columns:
        res = fit_one_pollutant(p, df)
        print(f"{p:6s} | MAE={res['MAE']:.2f} | RMSE={res['RMSE']:.2f}")
        results.append(res)

PM2.5  | MAE=9.38 | RMSE=16.03
PM10   | MAE=13.92 | RMSE=23.24
SO2    | MAE=1.76 | RMSE=3.41
NO2    | MAE=6.29 | RMSE=10.53
CO     | MAE=191.86 | RMSE=382.93
O3     | MAE=7.88 | RMSE=11.74
